# Lab | Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In terminal run this:

`pip install ipykernel -U --force-reinstall`

`pip install python-dotenv`

In [11]:
import warnings
warnings.filterwarnings('ignore') # Hide all warnings, don’t show them in the output

In [16]:
from google.colab import userdata
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

#OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
# Set it as an environment variable, which LangChain often uses by default
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY
print("OpenAI API key loaded and set as environment variable.")
HUGGINGFACEHUB_API_TOKEN = os.getenv('HUGGINGFACEHUB_API_TOKEN')

OpenAI API key loaded and set as environment variable.


In [3]:
!pip install pandas

In [12]:
import pandas as pd
df = pd.read_csv('/content/sample_data/Data.csv')

In [13]:
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...


## LLMChain

In [7]:
!pip install langchain langchain-openai python-dotenv

INFO: pip is looking at multiple versions of langchain-openai to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 6.8 MB/s eta 0:00:00


In [14]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain, SimpleSequentialChain, SequentialChain

#### 1. Setting Up the Brain (The LLM)

- **What it is**: You created an instance of a powerful AI model (ChatOpenAI) and called it llm (Large Language Model). Think of this as hiring a super-smart, fast research assistant.

- **`temperature=0`** : You told your assistant to be as factual and deterministic as possible. A low temperature means it will give the most likely, consistent answer every time you ask the same question. If you set it higher, it would be more creative and random.

In [30]:
#Replace None by your own value and justify
llm = ChatOpenAI(temperature=0)
# llm = ChatOpenAI(temperature=0)    # deterministic
# llm = ChatOpenAI(temperature=0.5)  # balanced
# llm = ChatOpenAI(temperature=1)    # very creative

In [20]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### 2. Writing a Reusable Question (The Prompt Template)

- **What it is**: Instead of writing a new question every time, you created a template. The {product} part is a placeholder, like a blank space in a form.

- **Why it's clever**: You can now use this same template to ask about any product just by filling in the blank.

In [48]:
# prompt = ChatPromptTemplate.from_template( #Write a query that would take a variable to describe any product
# )

prompt = ChatPromptTemplate.from_template(
    "What is {product} and what are its key features?"
) # product is supposed to be replaced with product names when it is used.

### 3. You Connect the Brain to the Question (The Chain)

In [49]:
chain = LLMChain(llm=llm, prompt=prompt) # Creates a chain that connects the LLM with the prompt template, so you can easily run it with different products.

### 4. You Finally Ask a Question!

In [50]:
#product = "Queen Size Sheet Set"
product = "Royal Enfield Hymalayan 450"
#product = "Yamaha XTX"
chain.run(product)

'Royal Enfield Himalayan 450 is a rugged and versatile adventure touring motorcycle designed for long-distance journeys and off-road adventures. It is an upgraded version of the standard Royal Enfield Himalayan with a larger engine displacement of 450cc.\n\nKey features of the Royal Enfield Himalayan 450 include:\n1. Powerful 450cc single-cylinder engine for superior performance and efficiency.\n2. Upgraded suspension system for better handling and comfort on rough terrains.\n3. Dual-channel ABS for improved safety and braking performance.\n4. Enhanced ground clearance for better off-road capabilities.\n5. Comfortable upright riding position for long hours in the saddle.\n6. Windscreen for wind protection during highway cruising.\n7. Spacious luggage rack for carrying your gear on long trips.\n8. LED headlight and tail light for better visibility.\n9. Stylish and rugged design with adventure touring aesthetics.\n\nOverall, the Royal Enfield Himalayan 450 is a capable and reliable adven

## SimpleSequentialChain

In [56]:
from langchain.chains import SimpleSequentialChain

In [57]:
llm = ChatOpenAI(temperature=0.9)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "Describe the {product} in one sentence."
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [58]:

# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Can you suggest a suitable emoji for this product description? {chain_one}"
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [59]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [63]:
overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
The Royal Enfield Himalayan 450 is a rugged and versatile adventure motorcycle designed to conquer any terrain with ease.
🏍️🌄🔝

> Finished chain.


'🏍️🌄🔝'

**Repeat the above twice for different products**

## SequentialChain

In [70]:
from langchain.chains import SequentialChain

In [80]:
llm = ChatOpenAI(temperature=0.9)


first_prompt = ChatPromptTemplate.from_template(
  #This prompt should translate a review
  "Translate the {review}"
  #"Translate the review: The coffe is bad."
)

chain_one = LLMChain(llm=llm, prompt=first_prompt,
                     output_key="translated_review" #Give a name to your output
                    )


In [81]:
second_prompt = ChatPromptTemplate.from_template(
    #Write a promplt to summarize a review
    "Summarize the {review}"
)

chain_two = LLMChain(llm=llm, prompt=second_prompt,
                     output_key="summarized_review" #give a name to this output
                    )


In [82]:
# prompt template 3: translate to english or other language
third_prompt = ChatPromptTemplate.from_template(
    # None
    "What language is this review written in? Return only the language name: {review}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="language"
                      )


In [83]:

# prompt template 4: follow up message that take as inputs the two previous prompts' variables
fourth_prompt = ChatPromptTemplate.from_template(
        #None
        #"Summarize the review: {second_prompt} and reply back to what language it is the review written {third_prompt}"
        "Summarize the review: {translated_review} and reply back to what language it is the review written: {language}"
)
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="follow_up"
                     )


In [84]:
# overall_chain: input= Review
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["review"],
    output_variables=["translated_review", "language", "follow_up"],
    verbose=True
)

In [87]:
review = df.Review[1]
overall_chain(review)



> Entering new SequentialChain chain...

> Finished chain.


{'review': 'I loved the waterproof sac, although the opening was made of a hard plastic. I don’t know if that would break easily. But I couldn’t turn my phone on, once it was in the pouch.',
 'translated_review': "J'ai adoré le sac imperméable, même si l'ouverture était en plastique dur. Je ne sais pas si cela se casserait facilement. Mais je n'ai pas pu allumer mon téléphone une fois qu'il était dans la pochette.",
 'language': 'English',
 'follow_up': "The review is written in French. The reviewer loved the waterproof bag, but found the hard plastic opening a potential issue in terms of durability. They also mentioned that they couldn't turn on their phone once it was inside the pouch."}

**Repeat the above twice for different products or reviews**

## Router Chain

In [88]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts,
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity.

Here is a question:
{input}"""

biology_template = """You are an excellent biologist. \
You have a deep understanding of living organisms, \
from the molecular and cellular level to entire ecosystems. \
You are skilled at observing patterns in nature, analyzing biological data, \
and explaining complex processes like evolution, genetics, physiology, and ecology. \
You can clearly communicate how life functions and adapts, \
and you make connections between different biological concepts \
to answer challenging questions.

Here is a question:
{input}"""

In [89]:
prompt_infos = [
    {
        "name": "physics",
        "description": "Good for answering questions about physics",
        "prompt_template": physics_template
    },
    {
        "name": "math",
        "description": "Good for answering math questions",
        "prompt_template": math_template
    },
    {
        "name": "History",
        "description": "Good for answering history questions",
        "prompt_template": history_template
    },
    {
        "name": "computer science",
        "description": "Good for answering computer science questions",
        "prompt_template": computerscience_template
    },
    {
        "name": "biology",
        "description": "Good for answering biology questions",
        "prompt_template": biology_template
    }
]

In [90]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [91]:
llm = ChatOpenAI(temperature=0)

In [92]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain

destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [93]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [94]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [95]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [96]:
chain = MultiPromptChain(router_chain=router_chain,
                         destination_chains=destination_chains,
                         default_chain=default_chain, verbose=True
                        )

In [99]:
chain.run("What is an apple?")



> Entering new MultiPromptChain chain...
biology: {'input': 'What is an apple?'}
> Finished chain.


'An apple is a type of fruit that grows on apple trees. It is typically round in shape with a stem at the top and a core containing seeds in the center. Apples come in a variety of colors, sizes, and flavors, depending on the specific variety of apple. They are a rich source of vitamins, minerals, and dietary fiber, making them a nutritious and delicious snack. Apples are also used in a wide range of culinary dishes, from pies and crisps to salads and sauces.'

In [97]:
chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


"Black body radiation refers to the electromagnetic radiation emitted by a perfect black body, which is an idealized physical body that absorbs all incident electromagnetic radiation and emits radiation at all frequencies. The radiation emitted by a black body depends only on its temperature and follows a specific distribution known as Planck's law. This type of radiation is important in understanding concepts such as thermal radiation and the behavior of objects at different temperatures."

In [100]:
chain.run("what is 2 + 2")



> Entering new MultiPromptChain chain...
math: {'input': 'what is 2 + 2'}
> Finished chain.


'Thank you for the compliment! The answer to the question "what is 2 + 2" is 4.'

In [101]:
chain.run("Why does every cell in our body contain DNA?")



> Entering new MultiPromptChain chain...
biology: {'input': 'Why does every cell in our body contain DNA?'}
> Finished chain.


"Every cell in our body contains DNA because DNA is the genetic material that carries the instructions for the development, functioning, and reproduction of all living organisms. DNA contains the information needed to build and maintain an organism, including the proteins that make up our cells and tissues. \n\nHaving DNA in every cell ensures that each cell has the necessary genetic information to carry out its specific functions and to replicate itself accurately during cell division. This ensures that the genetic information is passed on to the next generation of cells, maintaining the integrity and continuity of the organism's genetic code.\n\nAdditionally, DNA serves as a storage system for genetic information that can be accessed and utilized by cells as needed. This allows for the regulation of gene expression, the repair of damaged DNA, and the adaptation to changing environmental conditions.\n\nIn summary, every cell in our body contains DNA because it is essential for the pro

**Repeat the above at least once for different inputs and chains executions - Be creative!**